# Train a ConvNet using multiple GPUs

This notebook shows how to train a deep model using multiple GPUs. In this notebook, we use CIFAR-10 dataset and train a VGG-like deep model using 2 GPUs.

First, **please make sure that you are running this notebook with an environment that has at least 2 GPUs**.

Let's import necessary packages:

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import get_cifar10

# Dataset preparation

Chainer provides a utility to retrieve and handle the CIFAR-10/100 dataset. In this notebook, we use the dataset class provided by Chainer to use CIFAR-10.

In [2]:
batchsize = 128

train, test = get_cifar10()
train_iter = iterators.MultiprocessIterator(train, batchsize)
test_iter = iterators.MultiprocessIterator(test, batchsize, repeat=False, shuffle=False)

# Model definition

Let's define a deep model.

In [3]:
class VGG(chainer.ChainList):

    def __init__(self, n_class):
        super(VGG, self).__init__(
            ConvBlock(64),
            ConvBlock(64, True),
            ConvBlock(128),
            ConvBlock(128, True),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256, True),
            LinearBlock(),
            LinearBlock(),
            L.Linear(None, n_class)
        )

    def __call__(self, x):
        for f in self.children():
            x = f(x)
        return x
    
class ConvBlock(chainer.Chain):

    def __init__(self, n_ch, pool_drop=False):
        w = chainer.initializers.HeNormal()
        super(ConvBlock, self).__init__()
        with self.init_scope():
            self.conv = L.Convolution2D(None, n_ch, 3, 1, 1, nobias=True, initialW=w)
            self.bn = L.BatchNormalization(n_ch)
        self.pool_drop = pool_drop

    def __call__(self, x):
        h = F.relu(self.bn(self.conv(x)))
        if self.pool_drop:
            h = F.max_pooling_2d(h, 2, 2)
            h = F.dropout(h, ratio=0.25)
        return h

class LinearBlock(chainer.Chain):

    def __init__(self):
        w = chainer.initializers.HeNormal()
        super(LinearBlock, self).__init__()
        with self.init_scope():
            self.fc = L.Linear(None, 1024, initialW=w)

    def __call__(self, x):
        return F.dropout(F.relu(self.fc(x)), ratio=0.5)

Then instantiate the model and setup a optimizer.

In [4]:
model = L.Classifier(VGG(n_class=10))
optimizer = optimizers.Adam()
optimizer.setup(model)

# Create ParallelUpdater

To use multiple GPUs for training, use `ParallelUpdater` instead of `StarndardUpdater` for the updater given to a trainer. So what you need to do is just replacing `StandardUpdater` with `ParallelUpdater`. That's all!

In [5]:
devices = {
    'main': 0,
    'gpu1': 1,
}

updater = training.ParallelUpdater(
    train_iter, optimizer, devices=devices)

trainer = training.Trainer(
    updater, (100, 'epoch'), out='multi_gpu_result')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'val/main/loss', 'main/accuracy', 'val/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.Evaluator(test_iter, model, device=devices['main']), name='val')
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

trainer.run()

epoch       main/loss   val/main/loss  main/accuracy  val/main/accuracy  elapsed_time
1           2.03247     1.59413        0.259471       0.388153           31.5463       
2           1.48999     1.35446        0.44805        0.526108           61.2136       
3           1.2023      1.13322        0.570152       0.624407           91.5109       
4           0.990455    1.18442        0.653932       0.60265            121.103       
5           0.866919    0.895297       0.701686       0.727354           152.11        
6           0.761392    0.828588       0.740625       0.744858           182.07        
7           0.688458    0.738975       0.77062        0.783129           212.247       
8           0.610161    0.706257       0.798678       0.785898           242.723       
9           0.543755    0.627599       0.821851       0.80627            272.914       
10          0.493121    0.599087       0.839034       0.803006           302.935       
11          0.450194    0.555716  

Process Process-20:
Process Process-28:
Process Process-30:
Process Process-8:
Process Process-32:
Traceback (most recent call last):
Process Process-11:
Process Process-14:
Process Process-15:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-19:
Process Process-22:
Process Process-3:
Process Process-29:
Process Process-7:
Process Process-16:
Traceback (most recent call last):
Process Process-9:
Process Process-5:
Process Process-17:
Process Process-23:
Process Process-6:
Process Process-25:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process Process-31:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/pro

  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Process Process-10:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-34:
Process Process-40:
Process Process-52:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
Process Process-54:
Process Process-59:
Process Process-48:
Process Process-61:
Process 

  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
Traceback (most recent call last):
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:


  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/p

  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues

  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, index = in_queue.get()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/iterators/multiprocess_iterator.py", line 386, in _worker
    cnt, mem_index, in

  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/shunta/.pyenv/versions/anaconda3

KeyboardInterrupt: 

In [ ]:
from IPython.display import Image
Image('mnist_result/loss.png')
Image('mnist_result/accuracy.png')

# ChainerCV

[ChainerCV](https://github.com/chainer/chainercv) is a collection of tools to train and run neural networks for computer vision tasks using Chainer. It provides many kinds of models, their pre-trained weights, data augomentation algorithms, dataset abstraction, evaluation tools, etc.

In this notebook, let's use ChainerCV to apply some transformations to the training images for data augmentation.

In [ ]:
%%bash
pip install chainercv

Chainer's `TransformDataset` takes two arguments, the first one is a dataset object and the second one is a transformation function. Each datum in the dataset is transformed by using the transformation function. The function should take an input and perform some transformations using any kind of libraries or just array maniputation to the input ndarray, and then return the resulting transformed input.

Let's train the same model with the `TransformDataset` that apply

- Random cropping of (28, 28)-sized sub-region
- Random LR-flipping
- Random color augmentation

using `chainercv.transforms.random_crop`, `chainercv.transforms.random_flip`, and `chainercv.transforms.pca_lighting`, respectively.

In [ ]:
from chainercv import transforms

def transform(inputs):
    img, label = inputs
    img = transforms.random_crop(img, (28, 28))
    img = transforms.random_flip(img)
    img = transforms.pca_lighting(img, 0.1)
    return img, label
    
batchsize = 128

train, test = get_cifar10()

train = datasets.TransformDataset(train, transform)

train_iter = iterators.MultiprocessIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

model = L.Classifier(VGG(n_class=10))
optimizer = optimizers.Adam()
optimizer.setup(model)

devices = {
    'main': 0,
    'gpu1': 1,
}
updater = training.ParallelUpdater(
    train_iter, optimizer, devices=devices)

trainer = training.Trainer(
    updater, (100, 'epoch'), out='multi_gpu_result_2')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'val/main/loss', 'main/accuracy', 'val/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.Evaluator(test_iter, model, device=devices['main']), name='val')
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

trainer.run()

epoch       main/loss   val/main/loss  main/accuracy  val/main/accuracy  elapsed_time
